In [1]:
using Optim
using DataFrames;using CSV
using Pkg
using LinearAlgebra;
using JuliaZH;JuliaZH.set_mirror("PKU")#JuliaZH.generate_startup("BFSU")
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)
using Base.Threads;nthreads()
using Meteor.ExactDiag;
using JLD2;using FileIO
using DifferentialEquations

In [2]:
function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3,:)
end
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end
    
function coherence_state(alpha)
    m=exp(alpha*a₊(dim)-alpha'*a₋(dim))*n0(dim)
    return  m*m'
end
    
function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end
    
function equ_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    for i in 1:n
        c[i,i]=1/n    
    end
    return c
end
    
dim=4
η=0.084 
    
function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));
a=a₊(dim)+a₋(dim)    
a2=kron([0 0;1 0],exp(im*η*a))
a3=kron([0 1;0 0],exp(-im*η*a))
# a2=kron([0 0;1 0],im*η*a₊(dim))
# a3=kron([0 1;0 0],-im*η*a₋(dim))   
#经过尝试发现,符合脉冲并不精准,都保留下来就很不精确
    
σ₊=[0 0;1 0]
σ₋=[0 1;0 0]
σ_x=[0 1;1 0]
σ_y=[0 -1im;1im 0]
σ_z=[1 0;0 -1]

function dm2vec(dm)
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,Int(sqrt(size(vec)[1])),:)
end
    
function temperature(nbar)
    return (6.626*1e-34*1e6*1)/(1.38*1e-23*log(1/nbar+1))
end
    
n=kron(I(2),a₊(dim)*a₋(dim))
en=kron([0 0;0 1],I(dim))

ν=1.
    
function H(Ω,Δ)
    unitary::Matrix{ComplexF64}=kron(I(dim*2),-im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3))+kron(transpose(im*(ν*n-Δ*en+Ω/2*a2+Ω'/2*a3)),I(dim*2))
    return unitary
end
    
function Hv(Ω,Δ)
    unitary::Matrix{ComplexF64}=ν*n-Δ*en+Ω/2*a2+Ω'/2*a3
    return unitary
end


Hv (generic function with 1 method)

In [3]:
instate=kron([1. 0;0 0],equ_state(dim,2));

In [4]:
function R(t,phi,omega,delta)
    return exp(-im*t*Hv(omega*exp(im*phi),delta))
end

R (generic function with 1 method)

In [5]:
function swap_up(dims,n)#下态到上态就行
    
    s=zeros(dims,dims)+I(dims)
    s[n+1,n+1]=0.;
   
    
    return s
end
swap=swap_up(dim,1)

4×4 Matrix{Float64}:
 1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0

In [6]:
# function swap_prod(dims,n)#蓝边带交换
#     s=zeros(dim*2,dim*2)+I(dim*2)
#     s[n+1,n+1]=0.;
#     s[n+2+dims,n+2+dims]=0.;
#     s[n+2+dims,n+1]=1.;
#     s[n+1,n+2+dims]=1.;
    
    
#     return s
# end
# swap=swap_prod(dim,2)

In [7]:
function feval(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    opres=norm(norm.(swapfit[1:dim,1:dim])-norm.(swap))
    println(round.(x,digits=5),"    ",round(opres,digits=4))    
    return opres
end

feval (generic function with 1 method)

In [33]:
xini=[1.72659, 145.2258, 149.34361, 144.09456, 5.44969, 6.28319]
# xin=[1.72755, 145.29433, 149.08461, 144.1136, 5.57726, 6.57151]#from1
# xini=res.minimizer
lower=[0.,0,0,0,0,0];upper=[3.,4*pi/η,4*pi/η,4*pi/η,2*pi,2*pi];
# res=optimize(feval,xini,ParticleSwarm(; lower = lower,upper = upper,n_particles = 10),Optim.Options(iterations = 10000))
res= optimize(feval,xini,LBFGS()) 

[1.7266, 145.2258, 149.34361, 144.09456, 5.44969, 6.28319]    0.3505
[1.72658, 145.2258, 149.34361, 144.09456, 5.44969, 6.28319]    0.3505
[1.72659, 145.22668, 149.34361, 144.09456, 5.44969, 6.28319]    0.3505
[1.72659, 145.22492, 149.34361, 144.09456, 5.44969, 6.28319]    0.3505
[1.72659, 145.2258, 149.34451, 144.09456, 5.44969, 6.28319]    0.3505
[1.72659, 145.2258, 149.34271, 144.09456, 5.44969, 6.28319]    0.3505
[1.72659, 145.2258, 149.34361, 144.09543, 5.44969, 6.28319]    0.3505
[1.72659, 145.2258, 149.34361, 144.09369, 5.44969, 6.28319]    0.3505
[1.72659, 145.2258, 149.34361, 144.09456, 5.44972, 6.28319]    0.3505
[1.72659, 145.2258, 149.34361, 144.09456, 5.44966, 6.28319]    0.3505
[1.72659, 145.2258, 149.34361, 144.09456, 5.44969, 6.28323]    0.3505
[1.72659, 145.2258, 149.34361, 144.09456, 5.44969, 6.28315]    0.3505
[1.72659, 145.2258, 149.34361, 144.09456, 5.44969, 6.28319]    0.3505
[1.14152, 145.22253, 149.33976, 144.09234, 5.44881, 6.28748]    0.9429
[1.14151, 145.2225

[1.72658, 145.22579, 149.3436, 144.09456, 5.44969, 6.2832]    0.3505
[1.72656, 145.22579, 149.3436, 144.09456, 5.44969, 6.2832]    0.3505
[1.72657, 145.22667, 149.3436, 144.09456, 5.44969, 6.2832]    0.3505
[1.72657, 145.22491, 149.3436, 144.09456, 5.44969, 6.2832]    0.3505
[1.72657, 145.22579, 149.3445, 144.09456, 5.44969, 6.2832]    0.3505
[1.72657, 145.22579, 149.3427, 144.09456, 5.44969, 6.2832]    0.3505
[1.72657, 145.22579, 149.3436, 144.09543, 5.44969, 6.2832]    0.3505
[1.72657, 145.22579, 149.3436, 144.09368, 5.44969, 6.2832]    0.3505
[1.72657, 145.22579, 149.3436, 144.09456, 5.44972, 6.2832]    0.3505
[1.72657, 145.22579, 149.3436, 144.09456, 5.44966, 6.2832]    0.3505
[1.72657, 145.22579, 149.3436, 144.09456, 5.44969, 6.28324]    0.3505
[1.72657, 145.22579, 149.3436, 144.09456, 5.44969, 6.28316]    0.3505
[1.72657, 145.22579, 149.3436, 144.09456, 5.44969, 6.2832]    0.3505
[1.72658, 145.22576, 149.34356, 144.09454, 5.44968, 6.28323]    0.3505
[1.72656, 145.22576, 149.34356

[1.7266, 145.22314, 149.33871, 144.09292, 5.44963, 6.28741]    0.3505
[1.7266, 145.22138, 149.33871, 144.09292, 5.44963, 6.28741]    0.3505
[1.7266, 145.22226, 149.33962, 144.09292, 5.44963, 6.28741]    0.3505
[1.7266, 145.22226, 149.33781, 144.09292, 5.44963, 6.28741]    0.3505
[1.7266, 145.22226, 149.33871, 144.09379, 5.44963, 6.28741]    0.3505
[1.7266, 145.22226, 149.33871, 144.09205, 5.44963, 6.28741]    0.3505
[1.7266, 145.22226, 149.33871, 144.09292, 5.44966, 6.28741]    0.3505
[1.7266, 145.22226, 149.33871, 144.09292, 5.4496, 6.28741]    0.3505
[1.7266, 145.22226, 149.33871, 144.09292, 5.44963, 6.28745]    0.3505
[1.7266, 145.22226, 149.33871, 144.09292, 5.44963, 6.28738]    0.3505
[1.7266, 145.22226, 149.33871, 144.09292, 5.44963, 6.28741]    0.3505
[1.72663, 145.22166, 149.33576, 144.09362, 5.45123, 6.29122]    0.3505
[1.72661, 145.22166, 149.33576, 144.09362, 5.45123, 6.29122]    0.3505
[1.72662, 145.22254, 149.33576, 144.09362, 5.45123, 6.29122]    0.3505
[1.72662, 145.2207

[1.72675, 145.22159, 149.30463, 144.10629, 5.47363, 6.33581]    0.3504
[1.72675, 145.22159, 149.30463, 144.10629, 5.47363, 6.33585]    0.3504
[1.7268, 145.2239, 149.29646, 144.10702, 5.47838, 6.34787]    0.3503
[1.72678, 145.2239, 149.29646, 144.10702, 5.47838, 6.34787]    0.3503
[1.72679, 145.22477, 149.29646, 144.10702, 5.47838, 6.34787]    0.3503
[1.72679, 145.22302, 149.29646, 144.10702, 5.47838, 6.34787]    0.3504
[1.72679, 145.2239, 149.29736, 144.10702, 5.47838, 6.34787]    0.3503
[1.72679, 145.2239, 149.29556, 144.10702, 5.47838, 6.34787]    0.3503
[1.72679, 145.2239, 149.29646, 144.10789, 5.47838, 6.34787]    0.3504
[1.72679, 145.2239, 149.29646, 144.10615, 5.47838, 6.34787]    0.3503
[1.72679, 145.2239, 149.29646, 144.10702, 5.47842, 6.34787]    0.3503
[1.72679, 145.2239, 149.29646, 144.10702, 5.47835, 6.34787]    0.3503
[1.72679, 145.2239, 149.29646, 144.10702, 5.47838, 6.3479]    0.3503
[1.72679, 145.2239, 149.29646, 144.10702, 5.47838, 6.34783]    0.3503
[1.72679, 145.2239

[1.72757, 145.27761, 149.1224, 144.1014, 5.58889, 6.58247]    0.3498
[1.72757, 145.27849, 149.1233, 144.1014, 5.58889, 6.58247]    0.3498
[1.72757, 145.27849, 149.1215, 144.1014, 5.58889, 6.58247]    0.3498
[1.72757, 145.27849, 149.1224, 144.10227, 5.58889, 6.58247]    0.3498
[1.72757, 145.27849, 149.1224, 144.10052, 5.58889, 6.58247]    0.3498
[1.72757, 145.27849, 149.1224, 144.1014, 5.58893, 6.58247]    0.3498
[1.72757, 145.27849, 149.1224, 144.1014, 5.58886, 6.58247]    0.3498
[1.72757, 145.27849, 149.1224, 144.1014, 5.58889, 6.58251]    0.3498
[1.72757, 145.27849, 149.1224, 144.1014, 5.58889, 6.58243]    0.3498
[1.72757, 145.27849, 149.1224, 144.1014, 5.58889, 6.58247]    0.3498
[1.72758, 145.27866, 149.12203, 144.10133, 5.58901, 6.58305]    0.3498
[1.72756, 145.27866, 149.12203, 144.10133, 5.58901, 6.58305]    0.3498
[1.72757, 145.27954, 149.12203, 144.10133, 5.58901, 6.58305]    0.3498
[1.72757, 145.27778, 149.12203, 144.10133, 5.58901, 6.58305]    0.3498
[1.72757, 145.27866, 149

[1.72754, 145.29187, 149.09374, 144.10997, 5.57741, 6.56818]    0.3498
[1.72754, 145.29187, 149.09374, 144.11084, 5.57745, 6.56818]    0.3498
[1.72754, 145.29187, 149.09374, 144.11084, 5.57738, 6.56818]    0.3498
[1.72754, 145.29187, 149.09374, 144.11084, 5.57741, 6.56822]    0.3498
[1.72754, 145.29187, 149.09374, 144.11084, 5.57741, 6.56814]    0.3498
[1.72754, 145.29187, 149.09374, 144.11084, 5.57741, 6.56818]    0.3498
[1.72764, 145.31176, 149.02728, 144.12934, 5.57904, 6.59246]    0.3499
[1.72762, 145.31176, 149.02728, 144.12934, 5.57904, 6.59246]    0.3499
[1.72763, 145.31264, 149.02728, 144.12934, 5.57904, 6.59246]    0.3499
[1.72763, 145.31088, 149.02728, 144.12934, 5.57904, 6.59246]    0.3499
[1.72763, 145.31176, 149.02819, 144.12934, 5.57904, 6.59246]    0.3499
[1.72763, 145.31176, 149.02638, 144.12934, 5.57904, 6.59246]    0.3499
[1.72763, 145.31176, 149.02728, 144.13021, 5.57904, 6.59246]    0.3499
[1.72763, 145.31176, 149.02728, 144.12847, 5.57904, 6.59246]    0.3499
[1.727

 * Status: success (objective increased between iterations)

 * Candidate solution
    Final objective value:     3.497909e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 2.52e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.69e-10 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.60e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.46e-14 ≰ 0.0e+00
    |g(x)|                 = 8.51e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   1  (vs limit Inf)
    Iterations:    24
    f(x) calls:    89
    ∇f(x) calls:   89


In [21]:
# xini

In [34]:
res.minimizer

6-element Vector{Float64}:
   1.72755063026237
 145.29432506937067
 149.08460912190097
 144.11360436570817
   5.577261308916006
   6.571508743301507

In [35]:
res.minimum

0.3497909430451502

In [36]:
function feval2(x)
    swapfit=R(x[4],x[6],1,x[1])*R(x[3],x[5],1,x[1])*R(x[2],0,1,x[1])
    
    return swapfit
end

feval2 (generic function with 1 method)

In [37]:
round.(DataFrame(abs.(feval2(xini)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.7089,0.0214,0.0595,0.0041,0.0163,0.0498,0.7004,0.0099
2,0.0271,0.1348,0.0377,0.0252,0.0143,0.0482,0.0407,0.9873
3,0.0564,0.0765,0.9901,0.0251,0.0292,0.0268,0.0833,0.0397
4,0.0133,0.0525,0.0215,0.9966,0.0032,0.0524,0.0118,0.0202
5,0.0343,0.0024,0.0285,0.0032,0.9986,0.0055,0.0237,0.0138
6,0.0471,0.0205,0.0255,0.0523,0.0058,0.9953,0.0274,0.0509
7,0.6999,0.0319,0.083,0.0167,0.0377,0.0224,0.7059,0.0413
8,0.0011,0.9856,0.0773,0.0489,0.0022,0.0243,0.0337,0.1363


In [38]:
# round.(DataFrame(feval2(xini)/(feval2(xini)[1]/abs.(feval2(xini))[1]),:auto),digits=4)

In [39]:
round.(DataFrame(abs.(feval2(res.minimizer)),:auto),digits=4)

Row,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.7113,0.0205,0.0597,0.0085,0.0122,0.0444,0.6984,0.0102
2,0.0273,0.1385,0.0382,0.0302,0.011,0.0506,0.0377,0.9867
3,0.0496,0.0766,0.9902,0.0245,0.0286,0.0268,0.0873,0.0364
4,0.0107,0.0554,0.0215,0.9968,0.0007,0.0466,0.0054,0.024
5,0.0374,0.0045,0.0281,0.0007,0.9985,0.0053,0.0243,0.0112
6,0.0407,0.0244,0.0257,0.0464,0.0059,0.9956,0.027,0.0541
7,0.6983,0.023,0.0826,0.0087,0.0423,0.0215,0.7079,0.0403
8,0.0039,0.9851,0.0752,0.051,0.0069,0.029,0.0283,0.1404
